In [2]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import spacy
# ! python -m spacy download it_core_news_sm
nlp = spacy.load("it_core_news_sm")
from sklearn.preprocessing import MinMaxScaler

In [59]:
df = pd.read_csv("ilpost.csv", parse_dates=["date"])

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11441 entries, 0 to 11440
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   headline  11441 non-null  object        
 1   summary   11441 non-null  object        
 2   category  11441 non-null  object        
 3   tag       11398 non-null  object        
 4   date      11441 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 447.0+ KB


In [60]:
df.dropna(inplace=True)

## Data Preprocessing

In [61]:
#making category and tag as type list
df["category"] = df["category"].apply(lambda x: x.split(","))
df["tag"] = df["tag"].apply(lambda x: x.split(","))

In [62]:
df.head()

,headline,summary,category,tag,date
0,L’esercito continua a sparare cannonate nel ma...,Da oltre 20 anni le associazioni ambientalist...,[italia],"[esercito-italiano, pianeta, punta-bianca, sic...",2022-02-04
1,I dati sul coronavirus in Italia di giovedì 3 ...,,"[bits, italia]","[coronavirus, coronavirus-italia, covid-19]",2022-02-03
2,Da venerdì 4 febbraio in Italia sarà distribui...,,"[bits, italia]","[coronavirus, covid-19, paxlovid]",2022-02-03
3,Il comune di Napoli deve abbattere una scuola ...,Nel 2010 fu costruita senza permessi in un'ar...,[italia],"[camaldoli, chiaiano, luigi-de-magistris, napo...",2022-02-03
4,I dati della settimana su coronavirus e vaccin...,Il calo dei ricoveri in terapia intensiva e d...,[italia],"[coronavirus, covid-19, dati-coronavirus, vacc...",2022-02-03


In [63]:
#creating new feature corpus
df["corpus"] = df["headline"]+df["summary"]

In [64]:
df.head()["corpus"][0]

'L’esercito continua a sparare cannonate nel mare di Punta Bianca Da oltre 20 anni le associazioni ambientaliste chiedono di dismettere il poligono siciliano che si trova in mezzo a una riserva naturale'

In [65]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
def sentiment(text):
    """This function creates the sentiment score for each news

    Args:
        text (str): news text

    Returns:
        int: score from 1 to 5
    """
    tokens= tokenizer.encode(text, return_tensors="pt")
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [66]:
# df["sentiment"] = df["corpus"].apply(sentiment)
# df.to_csv("ipost_sentiment.csv")

In [3]:
df = pd.read_csv("ilpost_sentiment.csv", parse_dates=["date"])

In [4]:
#scaling sentiment score from -1 to 1
scaler = MinMaxScaler(feature_range=(-1,1))
df["sentiment"] = scaler.fit_transform(df["sentiment"].to_frame())

In [5]:
#creating lemmatized text feature
def text_cleaner(text):
    """
    This function preprocess the text returning a string.
    
    It applies the following steps in this order: 
    punctuation removal, tokenization, removal of stop words, lemmatization.
    
    Args:
        text (str): text to be preprocessed

    Returns:
        str: processed text
    """
    
    try:
        
        if type(text)==int or type(text)==float and not np.isnan(text):
            text=str(text)
            
        tokens = word_tokenize(text)
        stop_words = stopwords.words("italian")
        stop_words.extend(["’", "‘","“", "”", ",", ".", "(", ")", ":"])
        filtered_text = " ".join([word.rstrip().lstrip().lower() for word in tokens if word.lower() not in stop_words])
        filtered_text = nlp(filtered_text)
        lemmatized_text = " ".join([word.lemma_ for word in filtered_text ])
    
        return lemmatized_text
        
    except Exception as e:
        print("Sorry", e)
        return None

In [6]:
df["lemma_corpus"] = df["corpus"].apply(text_cleaner)

In [ ]:
df.head()

,headline,summary,category,tag,date,corpus,sentiment,lemma_corpus
0,"I dati sul coronavirus in Italia di oggi, giov...",Nelle ultime 24 ore in Italia sono stati rile...,"[bits, italia]","[coronavirus, coronavirus-italia, covid-19]",2022-02-03,"I dati sul coronavirus in Italia di oggi, giov...",-1.0,dato coronavirus italia oggi giovedì 3 febbrai...
1,Da venerdì 4 febbraio in Italia sarà distribui...,,"[bits, italia]","[coronavirus, covid-19, paxlovid]",2022-02-03,Da venerdì 4 febbraio in Italia sarà distribui...,-1.0,venerdì 4 febbraio italia distribuire paxlovid...
2,Il comune di Napoli deve abbattere una scuola ...,Nel 2010 fu costruita senza permessi in un'ar...,[italia],"[camaldoli, chiaiano, luigi-de-magistris, napo...",2022-02-03,Il comune di Napoli deve abbattere una scuola ...,-1.0,comune napoli dovere abbattere scuola nuovo 20...
3,I dati della settimana su coronavirus e vaccin...,Il calo dei ricoveri in terapia intensiva e d...,[italia],"[coronavirus, covid-19, dati-coronavirus, vacc...",2022-02-03,I dati della settimana su coronavirus e vaccin...,1.0,dato settimana coronavirus vaccinazione italia...
4,Cosa ha deciso il governo su colori delle regi...,Le regole per eventuali zone rosse riguardera...,[italia],"[coronavirus, covid-19, green-pass, terza-dose]",2022-02-02,Cosa ha deciso il governo su colori delle regi...,0.0,cosa deciso governo colore regione green pass ...
